# Validation of the PyPSA-Africa generation by hydro powerplants

This notebooks investigates the total energy generation by hydro powerplants by the workflow

To reproduce the findings obtained in this notebook,
please run the full snakemake workflow for the Africa.
To do so, please set ``countries = ["Africa"]`` in the ``config.yaml`` file.

Note. The solved network is required as the total generation dispatch of the optimized network is considered. 

## Load python imports

In [ ]:
import logging
import os

import pypsa
import yaml
import pandas as pd
import geopandas as gpd
import numpy as np
import scipy as sp
import networkx as nx

# plotting stuff
import matplotlib.pyplot as plt

plt.style.use("bmh")
import seaborn as sns  ###
import cartopy.crs as ccrs

sns.set(style="darkgrid")

from scipy.sparse import csgraph
from itertools import product

from shapely.geometry import Point, LineString
import shapely, shapely.prepared, shapely.wkt

logger = logging.getLogger(__name__)

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", 70)

import sys

sys.path.append("../../")  # to import helpers
from scripts._helpers import sets_path_to_root

sets_path_to_root("pypsa-africa")

max_node_size = 1.0  # maximum size of a node for plotting purposes [used in plots]

## Load network

Load the PyPSA sovled network

In [ ]:
network = os.getcwd() + "/results/networks/elec_s_10_ec_lcopt_Co2L-24H.nc"

n = pypsa.Network(network)

## Calculate total hydro energy production by powerplants

Total energy production by dams [TWh]

In [ ]:
hydroenergy_by_dams = n.storage_units_t.p.mean().sum() / 1000 * 8760
hydroenergy_by_dams

Total energy production by run-of-river [TWh]

In [ ]:
ror_gen_index = n.generators[n.generators.carrier == "ror"].index
hydroenergy_by_ror = n.generators_t.p[ror_gen_index].mean().sum() / 1000 * 8760
hydroenergy_by_ror

In [ ]:
n.storage_units_t.keys()

Total hydropower energy production [TWh]

In [ ]:
tot_hydroenergy = hydroenergy_by_dams + hydroenergy_by_ror
tot_hydroenergy

## Calculate Nigeria hydro energy production by powerplants

Get Nigerian hydropower generators by dam [TWh]

In [ ]:
country_su = n.storage_units_t.p.columns.str.split(" ", 1).str[0]
nigerian_columns_id_su = country_su == "NG"
nigerian_hydroenergy_dams = (
    n.storage_units_t.p.iloc[:, nigerian_columns_id_su].mean().sum() / 1000 * 8760
)
nigerian_hydroenergy_dams

Get Nigerian hydropower generators by run-of-river [TWh]

In [ ]:
country_ror_bool = ror_gen_index.str.split(" ", 1).str[0] == "NG"
nigerian_columns_id_ror = ror_gen_index[country_ror_bool]
nigerian_hydroenergy_ror = (
    n.generators_t.p.loc[:, nigerian_columns_id_ror].mean().sum() / 1000 * 8760
)
nigerian_hydroenergy_ror

In [ ]:
n.storage_units_t.inflow.mean().sum() * 8760

Total hydropower energy production in Nigeria [TWh]

In [ ]:
tot_nigerian_hydroenergy = nigerian_hydroenergy_dams + nigerian_hydroenergy_ror
tot_nigerian_hydroenergy

## Validation of the data with respect to IRENA

The data are compared to [IRENA 2020](https://www.irena.org/-/media/Files/IRENA/Agency/Publication/2020/Jul/IRENA_Renewable_Energy_Statistics_2020.pdf) and [IEA 2019](https://www.iea.org/regions/africa)
In particular, the following hydropower generation is expected:

| Source | Nigeria (2013) | Africa (2013) |
|--------|----------------|---------------|
| [IRENA 2020](https://www.irena.org/-/media/Files/IRENA/Agency/Publication/2020/Jul/IRENA_Renewable_Energy_Statistics_2020.pdf) | 6.12 TWh | 122.34 TWh |
| [IEA 2019](https://www.iea.org/regions/africa) | 5.33 TWh | 120.52 TWh |

Print PyPSA result

In [ ]:
print("| Source    |   Nigeria (2013)  |  Africa (2013) |")
print(
    "| PyPSA     |      %.2f TWh    |    %.2f TWh   |"
    % (tot_nigerian_hydroenergy, tot_hydroenergy)
)